In [90]:
import pandas as pd
import requests
import datetime as dt
from typing import List

In [6]:
def download_carga_horaria(ano_inicio: int, ano_fim: int, printer: bool=False) -> pd.DataFrame:
    """Função para fazer download dos dados de carga elétrica por subsistema no período de referência em base horária.
       Página: https://dados.ons.org.br/dataset/curva-carga

    Args:
        ano_inicio (int): ano inicial de extração.
        ano_fim (int): ano final de extração.
        printer(bool): rastreia ano que está sendo extraído com print em tela.

    Returns:
        pd.DataFrame: Pandas DataFrame com dados de carga horária entre ano_inicio e ano_fim, tendo como índice a data-hora.
    """

    url = "https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/curva-carga-ho/CURVA_CARGA_{}.csv"
    # verificar se anos inicial e final estão disponíveis
    get0 = requests.get(url.format(ano_inicio)).status_code # verify = False (autenticação)
    getn = requests.get(url.format(ano_fim)).status_code 
    if (get0 == 200) and (getn == 200): # 200: página (ano) disponível
        # concatenar arquivos de cada ano em um único dataframe
        df = pd.DataFrame()
        for ano in range(ano_inicio, ano_fim + 1):
            if printer:
                print(f"Lendo ano {ano}...")
            df2 = pd.read_csv(url.format(ano), sep = ";")
            df = pd.concat([df, df2])
        df.columns = ["id_reg", "desc_reg", "date", "load_mwmed"]
        df.loc[:, "date"] = pd.to_datetime(df.loc[:, "date"], format = '%Y-%m-%d %H:%M:%S')
        df.sort_values(by = "date", inplace = True)
        df.set_index("date", inplace=True)
        return df
    else:
       print("Ano não disponível.")

In [7]:
df = download_carga_horaria(2000, 2023, printer=True)

Lendo ano 2000...
Lendo ano 2001...
Lendo ano 2002...
Lendo ano 2003...
Lendo ano 2004...
Lendo ano 2005...
Lendo ano 2006...
Lendo ano 2007...
Lendo ano 2008...
Lendo ano 2009...
Lendo ano 2010...
Lendo ano 2011...
Lendo ano 2012...
Lendo ano 2013...
Lendo ano 2014...
Lendo ano 2015...
Lendo ano 2016...
Lendo ano 2017...
Lendo ano 2018...
Lendo ano 2019...
Lendo ano 2020...
Lendo ano 2021...
Lendo ano 2022...
Lendo ano 2023...


In [8]:
df

,id_reg,desc_reg,load_mwmed
date,,,
2000-01-01 00:00:00,N,NORTE,2373.7
2000-01-01 00:00:00,NE,NORDESTE,5340.2
2000-01-01 00:00:00,S,SUL,5777.0
2000-01-01 00:00:00,SE,SUDESTE,21183.0
2000-01-01 01:00:00,N,NORTE,2331.6
...,...,...,...
2023-02-26 19:00:00,S,SUL,0.0
2023-02-26 20:00:00,S,SUL,0.0
2023-02-26 21:00:00,S,SUL,0.0


In [13]:
def download_carga_diaria(ano_inicio: int, ano_fim: int, printer: bool=False):
    """ Função para fazer download dos dados de carga elétrica por subsistema no período de referência em base diária."""

    url = "https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/carga_energia_di/CARGA_ENERGIA_{}.csv"

    # verificar se anos inicial e final estão disponíveis
    get0 = requests.get(url.format(ano_inicio)).status_code # verify = False (autenticação)
    getn = requests.get(url.format(ano_fim)).status_code 
    if (get0 == 200) and (getn == 200): # 200: página (ano) disponível

        # concatenar arquivos de cada ano em um único dataframe
        df = pd.DataFrame()
        for ano in range(ano_inicio, ano_fim + 1):
            if printer:
                print(f"Lendo ano {ano}...")
            df2 = pd.read_csv(url.format(ano), sep = ";")
            df = pd.concat([df, df2])
        df.columns = ["id_reg", "desc_reg", "date", "load_mwmed"]
        df.loc[:, "date"] = pd.to_datetime(df.loc[:, "date"], format = '%Y-%m-%d')
        df.sort_values(by = "date", inplace = True)
        df.set_index("date", inplace=True)
        return df
    
    else:
       print("Ano não disponível.")

In [14]:
df = download_carga_diaria(2000, 2023, printer=True)

Lendo ano 2000...
Lendo ano 2001...
Lendo ano 2002...
Lendo ano 2003...
Lendo ano 2004...
Lendo ano 2005...
Lendo ano 2006...
Lendo ano 2007...
Lendo ano 2008...
Lendo ano 2009...
Lendo ano 2010...
Lendo ano 2011...
Lendo ano 2012...
Lendo ano 2013...
Lendo ano 2014...
Lendo ano 2015...
Lendo ano 2016...
Lendo ano 2017...
Lendo ano 2018...
Lendo ano 2019...
Lendo ano 2020...
Lendo ano 2021...
Lendo ano 2022...
Lendo ano 2023...


In [15]:
df

,id_reg,desc_reg,load_mwmed
date,,,
2000-01-01,N,NORTE,2243.512500
2000-01-01,NE,NORDESTE,4646.370833
2000-01-01,S,SUL,4800.650000
2000-01-01,SE,SUDESTE,19045.995833
2000-01-02,N,NORTE,2259.808333
...,...,...,...
2023-02-25,NE,NORDESTE,NaN
2023-02-26,NE,NORDESTE,NaN
2023-02-26,S,SUL,NaN


In [69]:
class ons_data:
    def __init__(self, freq: str, ano_inicio: int, ano_fim: int, idreg: str=None):
        self.freq = freq
        self.ano_inicio = ano_inicio
        self.ano_fim = ano_fim
        self.idreg = idreg
        self.data_dir = "../../../data/"

    def read(self) -> pd.DataFrame:
        """Função para ler arquivos "csv" já presentes no diretório de dados.

        Args:
            idreg (str): sub-região. ['N', 'NE', 'S', 'SE']

        Returns:
            pd.DataFrame: série de carga elétrica no período entre ano_inicio e ano_fim.
        """
        if self.freq == "hourly":
            path = "".join([self.data_dir,"hourly_load.csv"])
        elif self.freq == "daily":
            path = "".join([self.data_dir,"daily_load.csv"])
        df = pd.read_csv(path)
        if not self.idreg:
            idreg = df["id_reg"].unique()
        else:
            idreg = [self.idreg]
        df = df[df["id_reg"].isin(idreg)]
        df.set_index("date", inplace=True)
        return df

    def update(self, printer=False, write=False):
        if self.freq == "hourly":
            url = "https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/curva-carga-ho/CURVA_CARGA_{}.csv"
            date_format = "%Y-%m-%d %H:%M:%S"
        elif self.freq == "daily":
            url = "https://ons-dl-prod-opendata.s3.amazonaws.com/dataset/carga_energia_di/CARGA_ENERGIA_{}.csv"
            date_format = "%Y-%m-%d"
        else:
            raise Exception("Frequência não reconhecida. Utilize 'hourly' ou 'daily'.")
        get0 = requests.get(url.format(self.ano_inicio)).status_code # verify = False (autenticação)
        getn = requests.get(url.format(self.ano_fim)).status_code 
        if (get0 == 200) and (getn == 200): # 200: página (ano) disponível
            # concatenar arquivos de cada ano em um único dataframe
            df = pd.DataFrame()
            for ano in range(self.ano_inicio, self.ano_fim + 1):
                if printer:
                    print(f"Lendo ano {ano}...")
                df2 = pd.read_csv(url.format(ano), sep = ";")
                df = pd.concat([df, df2])
            df.columns = ["id_reg", "desc_reg", "date", "load_mwmed"]
            df.loc[:, "date"] = pd.to_datetime(df.loc[:, "date"], format = date_format)
            df.sort_values(by = "date", inplace = True)
            df.set_index("date", inplace=True)
            if write:
                full_path = "".join([self.data_dir,f"{self.freq}_load.csv"])
                df.to_csv(full_path)
            return df
        else:
            print("Ano não disponível.")

In [93]:
data = ons_data('hourly', 2000, 2023, idreg="S")

In [94]:
df = data.read()

In [95]:
df

,id_reg,desc_reg,load_mwmed
date,,,
2000-01-01 00:00:00,S,SUL,5777.000
2000-01-01 01:00:00,S,SUL,5580.700
2000-01-01 02:00:00,S,SUL,5098.700
2000-01-01 03:00:00,S,SUL,4753.700
2000-01-01 04:00:00,S,SUL,4584.100
...,...,...,...
2023-02-27 19:00:00,S,SUL,15408.276
2023-02-27 20:00:00,S,SUL,15247.607
2023-02-27 21:00:00,S,SUL,15027.497


In [169]:

def check_date_column(date_col: List[dt.datetime], _freq: str, printer=False) -> print:

    missing_dates = pd.date_range(date_col.min(), date_col.max(), freq=_freq).difference(date_col)
    missing_list = missing_dates.to_list()
    if printer:
        print("Datas faltantes:\n", missing_list)
    return missing_list

In [170]:
a = check_date_column(df.index, _freq="h")

Datas faltantes:
 [Timestamp('2000-10-08 00:00:00'), Timestamp('2001-10-14 00:00:00'), Timestamp('2002-11-03 00:00:00'), Timestamp('2003-10-19 00:00:00'), Timestamp('2004-11-02 00:00:00'), Timestamp('2005-10-16 00:00:00'), Timestamp('2006-11-05 00:00:00'), Timestamp('2007-10-14 00:00:00'), Timestamp('2008-10-19 00:00:00'), Timestamp('2009-10-18 00:00:00'), Timestamp('2010-10-17 00:00:00'), Timestamp('2011-10-16 00:00:00'), Timestamp('2012-10-21 00:00:00'), Timestamp('2013-10-20 00:00:00')]


In [168]:
a.to_list()

[Timestamp('2000-10-08 00:00:00'),
 Timestamp('2001-10-14 00:00:00'),
 Timestamp('2002-11-03 00:00:00'),
 Timestamp('2003-10-19 00:00:00'),
 Timestamp('2004-11-02 00:00:00'),
 Timestamp('2005-10-16 00:00:00'),
 Timestamp('2006-11-05 00:00:00'),
 Timestamp('2007-10-14 00:00:00'),
 Timestamp('2008-10-19 00:00:00'),
 Timestamp('2009-10-18 00:00:00'),
 Timestamp('2010-10-17 00:00:00'),
 Timestamp('2011-10-16 00:00:00'),
 Timestamp('2012-10-21 00:00:00'),
 Timestamp('2013-10-20 00:00:00')]

In [151]:
a = df.reset_index()
a[a["date"] == "2000-11-08 00:00:00"]

,date,id_reg,desc_reg,load_mwmed
7487,2000-11-08 00:00:00,S,SUL,6167.5


In [138]:
#dates = list(pd.date_range(df.index.min(), df.index.max(), freq="h"))
dates = pd.date_range(df.index.min(), df.index.max(), freq="h")
date_col = list(df.index)

In [149]:
pd.date_range(df.index.min(), df.index.max(), freq="h").difference(df.index)

DatetimeIndex(['2000-10-08', '2001-10-14', '2002-11-03', '2003-10-19',
               '2004-11-02', '2005-10-16', '2006-11-05', '2007-10-14',
               '2008-10-19', '2009-10-18', '2010-10-17', '2011-10-16',
               '2012-10-21', '2013-10-20'],
              dtype='datetime64[ns]', freq=None)

In [147]:
dates[dates=='2000-01-01 00:00:00']

DatetimeIndex(['2000-01-01'], dtype='datetime64[ns]', freq='H')